---
# Illusion Workshop Project II: Temporal Processing & Spatial Size Perception
---
#### Lea M. Hadzic, PSYCH 30 – Dec 3rd, 2025

This project investigates how temporal synchrony modulates size perception, testing the hypothesis that synchronized blinking strengthens perceptual grouping to enhance the distortion of a static central target. This notebook contains the complete Python and Pygame implementation of the illusion, featuring an interactive simulation that allows for the real-time manipulation of variables such as blink frequency, looming speed, and chromatic depth cues.

### 1. Imports

In [ ]:
import pygame
import math
import random
import sys

import matplotlib.pyplot as plt
import matplotlib.patches as patches

---

### 2. Configuration
Defines the static constants for the window resolution and the physical boundaries of the illusion, including the minimum and maximum sizes and distances for the "breathing" effect.

In [ ]:
WIDTH, HEIGHT = 800, 600
CENTER_RADIUS = 25     

# GEOMETRY
NUM_DOTS = 8           
MIN_DIST = 55         
MAX_DIST = 260
MIN_SURROUND_RADIUS = 5    
MAX_SURROUND_RADIUS = 75 

In [ ]:
# Setup the plot area
fig, ax = plt.subplots(figsize=(10, 3))
ax.set_xlim(0, 400)
ax.set_ylim(0, 160)
ax.set_aspect('equal') # Crucial so circles aren't ovals
ax.axis('off') # Hide the graph grid

# 1. Draw "Min" Outer Dot (The smallest the surround gets)
min_circle = patches.Circle((60, 80), MIN_SURROUND_RADIUS, color='black', label='Min Inducer')
ax.add_patch(min_circle)
ax.text(60, 80 - MIN_SURROUND_RADIUS - 20, "Min Surround\n(Close)", ha='center', fontsize=9)

# 2. Draw Center Target (The constant red circle)
target_circle = patches.Circle((160, 80), CENTER_RADIUS, color='black', label='Target')
ax.add_patch(target_circle)
ax.text(160, 80 - CENTER_RADIUS - 20, "Center Target\n(Fixed)", ha='center', fontsize=9)

# 3. Draw "Max" Outer Dot (The huge distant moon)
max_circle = patches.Circle((300, 80), MAX_SURROUND_RADIUS, color='black', label='Max Inducer')
ax.add_patch(max_circle)
ax.text(300, 80 - MAX_SURROUND_RADIUS - 20, "Max Surround\n(Far)", ha='center', fontsize=9)

plt.title(f"Geometry Scale Check: Inducers grow {MAX_SURROUND_RADIUS/MIN_SURROUND_RADIUS:.1f}x in size")
plt.show()

---

### 3. Color Themes
Stores the visual presets (e.g., "Science White," "Red & Blue") in a dictionary structure, allowing the user to cycle through different scientific and aesthetic modes during runtime.

In [ ]:
THEMES = [
    # 1. Red Center, Blue Surround (Depth Effect)
    {"name": "Red & Blue", "center": (255, 0, 0), "surround": (0, 0, 255), "bg": (0, 0, 0)},

    # 2. High Contrast White (Strongest Grouping)
    {"name": "Science White", "center": (200, 200, 200), "surround": (255, 255, 255), "bg": (0, 0, 0)},

    # 3. Cyan & Pink (Aesthetic)
    {"name": "Cyberpunk", "center": (255, 20, 147), "surround": (0, 255, 255), "bg": (0, 0, 0)},
]

In [ ]:
def norm_rgb(rgb):
    return (rgb[0]/255, rgb[1]/255, rgb[2]/255)

num_themes = len(THEMES)
fig, axes = plt.subplots(1, num_themes, figsize=(15, 4))

for i, theme in enumerate(THEMES):
    ax = axes[i]
    
    bg_color = norm_rgb(theme['bg'])
    ax.set_facecolor(bg_color)
    
    surround_color = norm_rgb(theme['surround'])
    surround_patch = patches.Circle((0.5, 0.5), 0.35, transform=ax.transAxes, 
                                    color=surround_color)
    ax.add_patch(surround_patch)

    center_color = norm_rgb(theme['center'])
    center_patch = patches.Circle((0.5, 0.5), 0.15, transform=ax.transAxes, 
                                  color=center_color)
    ax.add_patch(center_patch)
    

    ax.set_title(theme['name'], fontsize=11, weight='bold', pad=10)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.show()

---

### 4. Engine Setup
Initializes the Pygame library, creates the graphics window, loads the font renderer, and sets the starting values for all interactive variables (like frequency and speed).

In [ ]:
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Illusion Master Control")
font = pygame.font.SysFont("Consolas", 16, bold=True) # Monospace font for data
clock = pygame.time.Clock()
pygame.event.pump()

# Initial State
current_theme_index = 0
is_sync = True
blink_pattern = 'SQUARE' # 'SINE' or 'SQUARE'
blink_freq = 6.0       
move_speed = 0.4       
dot_phases = [0.0] * NUM_DOTS

running = True
start_ticks = pygame.time.get_ticks()

Helper functions below contain utility functions for mathematical operations, specifically linear interpolation (map_range) for scaling sizes and color brightness adjustment (get_color).

In [ ]:
# --- HELPERS ---
def get_color(base_color, intensity):
    r = int(base_color[0] * intensity)
    g = int(base_color[1] * intensity)
    b = int(base_color[2] * intensity)
    return (r, g, b)

def map_range(value, in_min, in_max, out_min, out_max):
    value = max(in_min, min(in_max, value)) 
    normalized = (value - in_min) / (in_max - in_min)
    return out_min + normalized * (out_max - out_min)

Visualizing the controls of the interactive tuning dashboard.

In [1]:
print("="*45)
print("   INTERACTIVE CONTROL DASHBOARD")
print("="*45)
print(f"{'KEY':<10} | {'FUNCTION'}")
print("-" * 45)
print(f"{'SPACE':<10} | Toggle Synchrony (Sync vs Random)")
print(f"{'C':<10} | Cycle Color Themes")
print(f"{'P':<10} | Switch Pattern (Sine vs Square)")
print(f"{'UP/DOWN':<10} | Adjust Blink Frequency")
print(f"{'LEFT/RIGHT':<10} | Adjust Looming Speed")
print("-" * 45)
print(f"{'ESC':<10} | Quit Simulation")
print("="*45)

   INTERACTIVE CONTROL DASHBOARD
KEY        | FUNCTION
---------------------------------------------
SPACE      | Toggle Synchrony (Sync vs Random)
C          | Cycle Color Themes
P          | Switch Pattern (Sine vs Square)
UP/DOWN    | Adjust Blink Frequency
LEFT/RIGHT | Adjust Looming Speed
---------------------------------------------
ESC        | Quit Simulation


---

### 5. Main Loop
Executes the frame-by-frame logic: it captures user keyboard input, calculates the physics for motion and blinking based on the current time, renders the graphics to the buffer, and updates the display.

In [ ]:
while running:
    # INPUT HANDLING
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            
            # --- CONTROLS ---
            
            # SPACE: Toggle Sync
            elif event.key == pygame.K_SPACE:
                is_sync = not is_sync
                if is_sync:
                    dot_phases = [0.0] * NUM_DOTS
                else:
                    dot_phases = [random.uniform(0, 2 * math.pi) for _ in range(NUM_DOTS)]

            # C: Cycle Colors
            elif event.key == pygame.K_c:
                current_theme_index = (current_theme_index + 1) % len(THEMES)
            
            # P: Toggle Pattern (Sine vs Square)
            elif event.key == pygame.K_p:
                blink_pattern = 'SINE' if blink_pattern == 'SQUARE' else 'SQUARE'

            # ARROWS: Blink Frequency
            elif event.key == pygame.K_UP: 
                blink_freq += 0.5
            elif event.key == pygame.K_DOWN: 
                blink_freq = max(0.5, blink_freq - 0.5)

            # LEFT/RIGHT: Movement Speed
            elif event.key == pygame.K_RIGHT: 
                move_speed += 0.1
            elif event.key == pygame.K_LEFT: 
                move_speed = max(0.0, move_speed - 0.1) # 0.0 stops movement

    t = (pygame.time.get_ticks() - start_ticks) / 1000.0
    
    # Get Theme
    theme = THEMES[current_theme_index]
    screen.fill(theme["bg"])

    # CALCULATE PHYSICAL MOVEMENT
    # This dictates the physical size/distance of the dots
    movement_wave = (math.sin(2 * math.pi * move_speed * t) + 1) / 2
    current_dist = map_range(movement_wave, 0, 1, MIN_DIST, MAX_DIST)
    current_radius = map_range(movement_wave, 0, 1, MIN_SURROUND_RADIUS, MAX_SURROUND_RADIUS)

    # DRAW SURROUND
    for i in range(NUM_DOTS):
        angle = (i / NUM_DOTS) * 2 * math.pi
        x = WIDTH/2 + current_dist * math.cos(angle)
        y = HEIGHT/2 + current_dist * math.sin(angle)

        # Calculate Blinking
        raw_wave = math.sin(2 * math.pi * blink_freq * t + dot_phases[i])
        
        # Apply Pattern Logic
        if blink_pattern == 'SINE':
            brightness = (raw_wave + 1) / 2  # Smooth fade
        else:
            brightness = 1.0 if raw_wave > 0 else 0.0 # Harsh on/off

        color = get_color(theme["surround"], brightness)
        pygame.draw.circle(screen, color, (int(x), int(y)), int(current_radius))

    # DRAW CENTER
    pygame.draw.circle(screen, theme["center"], (int(WIDTH/2), int(HEIGHT/2)), int(CENTER_RADIUS))

    # DRAW DASHBOARD
    # Draw a semi-transparent box for text
    ui_bg = pygame.Surface((320, 130))
    ui_bg.set_alpha(200)
    ui_bg.fill((20, 20, 20))
    screen.blit(ui_bg, (10, 10))

    # Render Text Lines
    lines = [
        f"THEME (C):  {theme['name']}",
        f"SYNC (SPC): {'ON' if is_sync else 'OFF (Random)'}",
        f"WAVE (P):   {blink_pattern}",
        f"BLINK (UP/DOWN):  {blink_freq:.1f} Hz",
        f"SPEED (L/R):  {move_speed:.1f} Hz",
    ]

    for idx, line in enumerate(lines):
        txt_surf = font.render(line, True, (200, 200, 200))
        screen.blit(txt_surf, (20, 20 + idx * 20))

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()

---

Correspondence to: `lea@cs.stanford.edu`